### State model of quadruped robots

This notebook exemplifies and implementation of the state model of quadruped robots presented in the [MIT Cheetah 3 Paper](https://dspace.mit.edu/bitstream/handle/1721.1/126619/IROS.pdf?sequence=2&isAllowed=y)

This model is used in an optimization algorithm that calculates optimal ground reaction forces used for quadruped locomotion.

Due to the switching between stance and swing phases it is difficult to describe the exact behavior of quadruped robots mathematically without introducing a lot of complexity. However, using the common assumption that the robot is equipped with light legs and with low inertia compared to the body, a linear model can be derived.[<sup>1</sup>](#fn1)

The controller model makes use of the linear relationship between the COM acceleration $\ddot{\textbf{p}}_\mathrm{C}$ and the body angular acceleration $\dot{\omega}_b$ and the ground reaction forces $\textbf{F}$ between the leg tips and the ground.

Assuming a rigid body[<sup>2</sup>](#fn2), forces acting on the feet influence the COM position:

$$
\textbf{f}_0 + \textbf{f}_1 + \textbf{f}_2 + \textbf{f}_3 = m\,(\ddot{\textbf{p}}_\mathrm{COM} + \textbf{g})
$$



The same thing can be applied to torques. Using the distance from the foottip $\textbf{p}_\mathrm{n}$ to the COM position $\textbf{p}_\mathrm{COM}$ as a "lever", the ground reaction forces change the body's angular velocity $\dot{\omega}_\mathrm{b}$.

In order to write this down in matrix form instead of vector notation, [skew-symmetric matrices](https://en.wikipedia.org/wiki/Skew-symmetric_matrix#Cross_product) are used:
$$
\textbf{M} = \textbf{r}\times\textbf{f} = [\textbf{r}]_{\times}\,\textbf{f}
$$

where

$$
[\textbf{r}]_{\times} = 
\left(\begin{array}{rrr} 
0 & -r_3 & r_2 \\ 
r_3 & 0 & -r_1 \\ 
-r_2 & r_1 & 0 \\ 
\end{array}\right)
$$

Using this notation, the momentum is calculated with:

$$
\sum_{n=0}^N [\textbf{p}_\mathrm{n} - \textbf{p}_\mathrm{COM}]_{\times}\, \textbf{f}_\mathrm{n} = \textbf{I}_\mathrm{G}\,\dot{\omega}_\mathrm{b}
$$

Where $m$ is the robot's total mass and $\textbf{I}_\mathrm{G}$ is the rotational inertia.

Introducing a vector containing the forces on all four legs

$$
\textbf{f} = (\textbf{f}_0^T, \,\textbf{f}_1^T, \,\textbf{f}_2^T, \,\textbf{f}_3^T)^T
$$

The whole state model is:

$$
\underbrace{
    \left( \begin{array}{ccc}
    \textbf{I}_{3\times3} & ... & \textbf{I}_{3\times3} \\ 
    [\textbf{p}_0 - \textbf{p}_{\mathrm{COM}}]_{\times} & ... & [\textbf{p}_3 - \textbf{p}_{\mathrm{COM}}]_{\times} \\ 
    \end{array}\right)
}_\textbf{A}
\textbf{ f}
=
\underbrace{
    \left( \begin{array}{c}
    m\,(\ddot{\textbf{p}}_C + \textbf{g}) \\
    \textbf{I}_\mathrm{G}\,\dot{\omega}_\mathrm{b} \\ 
    \end{array}\right)
}_\textbf{b}
$$

with $\textbf{I}_{3\times3}$ being a 3x3 identity matrix

#### Example
To see how a Python implementation could look like, let's make an example where the robot balances on two diagonal feet (and the others are in air)

In [1]:
import numpy as np

# setting constants
g = np.array([0, 0, 9.81])
m = 1
Ig = np.eye(3)
I3 = np.eye(3)

# function to generate skew symmetric matrices
def skew(r):
    '''return the skew symmetric matrix corresponding to r'''
    return np.array([[    0, -r[2],  r[1]],
                     [ r[2],     0, -r[0]],
                     [-r[1],  r[0],    0]])

# leg and COM location
p0 = np.array([ 0.09,  0.04, 0.18]) # support
p1 = np.array([ 0.09, -0.04, 0.14]) # swing
p2 = np.array([-0.09,  0.04, 0.14]) # swing
p3 = np.array([-0.09, -0.04, 0.18]) # support
pc = np.array([0, 0, 0])

# generate A
A = np.vstack((np.hstack((         I3,          I3,         I3,          I3)),
               np.hstack((skew(p0-pc), skew(p1-pc), skew(p2-pc), skew(p3-pc))))
             )

# forces
f0 = 0.5*m*g
f1 = np.array([0, 0, 0])
f2 = np.array([0, 0, 0])
f3 = 0.5*m*g
f = np.hstack((f0, f1, f2, f3))

print("b = ", A@f)

b =  [0.   0.   9.81 0.   0.   0.  ]


As we can see in this example, the only acting acceleration on the body is gravity. Furthermore, there exists no angular acceleration. If we play around with the values and f.e. change the supporting legs to be on one side, the angular acceleration would become nonzero because in reality the robot would tip over

#### Sources:
- Bledt, Gerado et al: MIT Cheetah 3: Design and Control of a Robust, Dynamic Quadruped Robot
- Focchi, michele et al.: High-slope terrain locomotion for torque-controlled quadruped robots
- https://en.wikipedia.org/wiki/Skew-symmetric_matrix#Cross_product

#### Footnotes

<sup>1</sup>: <span id="fn1">This assumption does not hold for the SpotMicro framework, as the servo motors are included in the legs and make out the main part of the weight. Concluding, the COM changes constantly by a fair amount and the leg inertia cannot be neglected. This issue must be tackled before implementing the given model into the control software.</span>

<sup>2</sup>: <span id="fn2">A rigid body model implies than no mechanical part deforms if forces or torques are exerted on it. Again, this assumptions does not completely hold for SpotMicro as there an not negelectable amount of backlash present in the servos. However, this effect cannot be avoided with better calculations but only using a better mechanical construction.</span>